# Setup


In [2]:
from dotenv import load_dotenv

import os

load_dotenv()
API_KEY = os.getenv("OPENAI_KEY")




# Define an AutoGen agent

In [5]:
from autogen import ConversableAgent
llm_config = {"model": "gpt-3.5-turbo", "api_key": API_KEY}

In [7]:
agent = ConversableAgent(
    name="chatbot",
    llm_config = llm_config,
    human_input_mode="NEVER",
)

In [10]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)

print(reply)

Sure, here's one for you: Why couldn't the bicycle stand up by itself? Because it was two tired!


In [11]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Sure, which joke would you like me to repeat?


Agents by themselfs do not retain the context

## Conversation

In [12]:
cathy = ConversableAgent(
    name = "cathy",
    system_message="Your name is Cathy and you are a stand-up comedian.",
    llm_config = llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name = "joe",
    system_message="Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config = llm_config,
    human_input_mode="NEVER",
)


In [14]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I´m Joe. Cathy, let´s keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I´m Joe. Cathy, let´s keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Let's do this! So I recently joined a cooking class because I wanted to learn how to make gourmet meals. Turns out, the only thing I learned was how to order takeout more efficiently!

--------------------------------------------------------------------------------
joe (to cathy):

Well, at least now you can impress your friends with your top-notch phone ordering skills!

--------------------------------------------------------------------------------
cathy (to joe):

Exactly! My friends are always in awe of how quickly I can recite my order without stumbling. Who needs cooking skills when you have speed dial, am I right?

--------------------------------------------------------------------------------


Visualize:
1) Chat History
2) Cost
3) Summary of the conversation

In [15]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': 'I´m Joe. Cathy, let´s keep the jokes rolling.',
  'role': 'assistant'},
 {'content': "Hey Joe! Let's do this! So I recently joined a cooking class "
             'because I wanted to learn how to make gourmet meals. Turns out, '
             'the only thing I learned was how to order takeout more '
             'efficiently!',
  'role': 'user'},
 {'content': 'Well, at least now you can impress your friends with your '
             'top-notch phone ordering skills!',
  'role': 'assistant'},
 {'content': 'Exactly! My friends are always in awe of how quickly I can '
             'recite my order without stumbling. Who needs cooking skills when '
             'you have speed dial, am I right?',
  'role': 'user'}]


In [16]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 541,
                                                             'cost': 0.004342,
                                                             'prompt_tokens': 7061,
                                                             'total_tokens': 7602},
                                      'total_cost': 0.004342},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 636,
                                                             'cost': 0.0046045,
                                                             'prompt_tokens': 7301,
                                                             'total_tokens': 7937},
                                      'total_cost': 0.0046045}}


In [17]:
pprint.pprint(chat_result.summary)

('Exactly! My friends are always in awe of how quickly I can recite my order '
 'without stumbling. Who needs cooking skills when you have speed dial, am I '
 'right?')


## Get a better summary of the conversation

by using an LLM 

In [18]:
chat_result = joe.initiate_chat(
    cathy,
    message="I´m Joe. Cathy, let´s keep the jokes rolling.",
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_prompt="Write a summary of the conversation.",
)

joe (to cathy):

I´m Joe. Cathy, let´s keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Let's do this! So I recently joined a cooking class because I wanted to learn how to make gourmet meals. Turns out, the only thing I learned was how to order takeout more efficiently!

--------------------------------------------------------------------------------
joe (to cathy):

Well, at least now you can impress your friends with your top-notch phone ordering skills!

--------------------------------------------------------------------------------
cathy (to joe):

Exactly! My friends are always in awe of how quickly I can recite my order without stumbling. Who needs cooking skills when you have speed dial, am I right?

--------------------------------------------------------------------------------


In [19]:
pprint.pprint(chat_result.summary)

('Joe and Cathy enjoy making jokes. Cathy jokes about joining a cooking class '
 'and only learning to order takeout efficiently. Joe jokes about impressing '
 'friends with his phone ordering skills.')


End the conversation by asking the agent to provide an output message:

In [20]:
cathy = ConversableAgent(
    name = "cathy",
    system_message="Your name is Cathy and you are a stand-up comedian."
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config = llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name = "joe",
    system_message="Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config = llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"] ,
)

In [21]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I´m Joe. Cathy, let´s keep the jokes rolling."
)

joe (to cathy):

I´m Joe. Cathy, let´s keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Are you ready to laugh your heart out?

--------------------------------------------------------------------------------
joe (to cathy):

Hey Cathy! I'm always ready to laugh. Hit me with your best joke!

--------------------------------------------------------------------------------
cathy (to joe):

Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic one! I love a good pun. Here's one for you: Why don't skeletons fight each other? They don't have the guts!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's a good one, Joe! You've got some killer jokes up your sleeve!

--------------------------------------------

Retain the conversation memory:

In [22]:
cathy.send(message="What´s the last joke we talked about?", recipient=joe)

cathy (to joe):

What´s the last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was "Why did the scarecrow win an award? Because he was outstanding in his field!"

--------------------------------------------------------------------------------
cathy (to joe):

Thank you. Have a great day!

--------------------------------------------------------------------------------
joe (to cathy):

You too! Take care!

--------------------------------------------------------------------------------
cathy (to joe):

Goodbye!

--------------------------------------------------------------------------------
